In [1]:
import math
import csv
import cv2
import keras
import numpy as np
from keras import backend as K
from keras.optimizers import Adam
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.applications.mobilenet import preprocess_input
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from mobilenet_sipeed.mobilenet import MobileNet

Using TensorFlow backend.


In [2]:
def dev_image_generator(input_feed,data_folder, bs, mode="train", aug=None):
    index = 0
    
    NUM_IMAGES = len(input_feed)
    
    while True:
        
        # initialize our batches of images and labels
        images = []
        poly_cof = []
        
        # keep looping until we reach our batch size
        while len(images) < bs:
            
            line = input_feed[index]
            index = index + 1
            if index >= len(input_feed):
                index = 0;
            
            center_img_path = line[0]
            file_name = center_img_path
#             print('./{}/{}'.format(folder, file_name))
            
            if len(file_name) > 0:
                image = cv2.imread(file_name)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image,(96,96))
                
                if image is not None:
                    pixels = np.array(image,dtype='f')
#                     pixels = pixels.astype('float32')
                    pixels = pixels/255
                    images.append(pixels)
    #                 images.append(np.fliplr(image))

                    x0 = (float(line[1])+float(line[3]))/2
                    y0 = (float(line[2])+float(line[4]))/2
                    x1 = (float(line[5])+float(line[7]))/2
                    y1 = (float(line[6])+float(line[8]))/2
                    x2 = float(line[9])
                    y2 = float(line[10])
                    
                    x0x1 = math.sqrt(math.pow(x0-x1, 2)+math.pow(y0-y1, 2))
                    x0x2 = math.sqrt(math.pow(x2-x0, 2)+math.pow(y2-y0, 2))
                    x1x2 = math.sqrt(math.pow(x2-x1, 2)+math.pow(y2-y1, 2))
#                     tongg = line[1:]

#                     tongg2 = np.array(tongg,dtype='f')
# #                     tongg = tongg.astype(np.float)
#                     tongg2 = tongg2/128
# #                     tongg2 = tongg2*0.75
#                     tongg2 = tongg2.tolist()
                    poly_cof.append([x0x1/128,x0x2/128,x1x2/128])
               
        
        # if the data augmentation object is not None, apply it
        if aug is not None:
            (images, labels) = next(aug.flow(np.array(images),
                labels, batch_size=bs))
        # yield the batch to the calling function
#         print(poly_cof)
        yield (np.array(images), np.array(poly_cof))
    

In [3]:
## Reading all data and sepperate to train dataset and valdidate dataset ##

image_load_data = []
image_load_data_v = []

with open('data_out.csv') as f:

    reader = csv.reader(f)
    for csv_line in reader:
        if len(csv_line)>1 :
            image_load_data.append(csv_line)

with open('data_out_v.csv') as f:

    reader = csv.reader(f)
    for csv_line in reader:
        if len(csv_line)>1 :
            image_load_data_v.append(csv_line)
        
image_load_data = shuffle(image_load_data)
image_load_data_v = shuffle(image_load_data_v)
# data_train, data_test  = train_test_split(image_load_data,test_size=0.2)


In [4]:
poly_cof=[]
image_load_data_v[0]
tongg = image_load_data_v[3][1:]

tongg = np.array(tongg,dtype='f')
# tongg = tongg.astype(np.float)
tongg = tongg/128
tongg = tongg*0.75
tongg = tongg.tolist()
poly_cof.append(tongg)
print(poly_cof)

[[0.603515625, 0.369140625, 0.509765625, 0.36328125, 0.28125, 0.3515625, 0.369140625, 0.357421875, 0.41015625, 0.52734375]]


In [5]:
images = []
image = cv2.imread(image_load_data_v[0][0])
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image = cv2.resize(image,(96,96))
pixels = np.array(image,dtype='f')
pixels = pixels/255
images.append(pixels)
pixels.shape

(96, 96)

In [6]:
# poly_cof

In [4]:
# initialize the number of epochs to train for and batch_size
NUM_EPOCHS = 10
BATCH_SIZE = 16

# initialize the total number of training and testing image
NUM_TRAIN_IMAGES = len(image_load_data)
NUM_TEST_IMAGES = len(image_load_data_v)
ft = 'img'
fv='img_v'
trainGen = dev_image_generator(image_load_data,ft, BATCH_SIZE, mode = "train", aug = None)
testGen = dev_image_generator(image_load_data_v,fv, BATCH_SIZE, mode = "train", aug = None)

In [5]:
# initialize our Keras model and compile it

base_model=MobileNet(input_shape=(96, 96, 3), alpha = 0.25,depth_multiplier = 1, dropout = 0.001,include_top = False, weights = None, classes = 1000, backend=keras.backend, layers=keras.layers,models=keras.models,utils=keras.utils)

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dropout(0.2)(x)
x=Dense(15,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
# x=Dropout(0.1)(x)
# x=Dense(50,activation='relu')(x) #dense layer 3
preds=Dense(3)(x) #final layer with softmax activation
model=Model(inputs=base_model.input,outputs=preds)
# from keras.models import Sequential
# from keras.layers import Convolution2D, MaxPooling2D, Dropout
# from keras.layers import Flatten, Dense


# ## TODO: Specify a CNN architecture
# # Your model should accept 96x96 pixel graysale images in
# # It should have a fully-connected output layer with 30 values (2 for each facial keypoint)

# model = Sequential()
# model.add(Convolution2D(32, (5, 5), input_shape=(96,96,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Convolution2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))

# model.add(Convolution2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))

# model.add(Convolution2D(256, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))

# model.add(GlobalAveragePooling2D())

# model.add(Dense(128, activation='relu'))
# # model.add(Dense(256, activation='relu'))
# # model.add(Dense(128, activation='relu'))
# # model.add(Dense(64, activation='relu'))
# model.add(Dense(3))


# Summarize the model
model.summary()
# model.summary()

filepath="w-imp-{epoch:02d}-{val_loss:.5f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.compile(optimizer='adam', loss='mse')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 98, 98, 3)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 48, 48, 8)         216       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 48, 48, 8)         32        
_________________________________________________________________
conv1_relu (ReLU)            (None, 48, 48, 8)         0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 48, 48, 8)         72        
____________________________________________

In [6]:
# train the network

print("[INFO] training w/ generator...")
H = model.fit_generator(trainGen, steps_per_epoch=NUM_TRAIN_IMAGES // BATCH_SIZE, 
                        validation_data=testGen, validation_steps=NUM_TEST_IMAGES // BATCH_SIZE, 
                        epochs=NUM_EPOCHS, callbacks=callbacks_list)


[INFO] training w/ generator...

Epoch 1/10
3709/3709 [==============================] - 213s 57ms/step - loss: 0.0071 - val_loss: 0.0043

Epoch 00001: val_loss improved from inf to 0.00429, saving model to w-imp-01-0.00429.h5
Epoch 2/10
3709/3709 [==============================] - 225s 61ms/step - loss: 0.0045 - val_loss: 0.0052

Epoch 00002: val_loss did not improve from 0.00429
Epoch 3/10
3709/3709 [==============================] - 223s 60ms/step - loss: 0.0044 - val_loss: 0.0047

Epoch 00003: val_loss did not improve from 0.00429
Epoch 4/10
3709/3709 [==============================] - 222s 60ms/step - loss: 0.0043 - val_loss: 0.0034

Epoch 00004: val_loss improved from 0.00429 to 0.00341, saving model to w-imp-04-0.00341.h5
Epoch 5/10
3709/3709 [==============================] - 225s 61ms/step - loss: 0.0042 - val_loss: 0.0055

Epoch 00005: val_loss did not improve from 0.00341
Epoch 6/10
 161/3709 [>.............................] - ETA: 2:56 - loss: 0.0043

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\ER9X\Anaconda3\envs\yolo\lib\site-packages\IPython\core\interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-b59bca69f97c>", line 6, in <module>
    epochs=NUM_EPOCHS, callbacks=callbacks_list)
  File "C:\Users\ER9X\Anaconda3\envs\yolo\lib\site-packages\keras\legacy\interfaces.py", line 91, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\ER9X\Anaconda3\envs\yolo\lib\site-packages\keras\engine\training.py", line 1732, in fit_generator
    initial_epoch=initial_epoch)
  File "C:\Users\ER9X\Anaconda3\envs\yolo\lib\site-packages\keras\engine\training_generator.py", line 220, in fit_generator
    reset_metrics=False)
  File "C:\Users\ER9X\Anaconda3\envs\yolo\lib\site-packages\keras\engine\training.py", line 1514, in train_on_batch
    outputs = self.train_function(ins)
  File "C:\Users\ER9X\Anaconda3\envs\yolo\lib\site-packages\tensorflow_core\python\keras\

KeyboardInterrupt: 

In [ ]:
model.save('road_following_model.h5')

In [19]:
from keras.models import load_model

In [20]:
model = load_model("w-imp-09-0.00697.h5")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [7]:
import cv2
import dlib
from IPython.display import clear_output, display
import ipywidgets
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])
detector = dlib.get_frontal_face_detector()

    
image_widget = ipywidgets.Image(format='jpeg')
image_widget2 = ipywidgets.Label('data')
image_widget2.value = 'value'

display(image_widget,image_widget2)

Image(value=b'', format='jpeg')

Label(value='value')

In [19]:
crop_img = cv2.imread('3.jpg')
# crop_img = cv2.resize(crop_img,(128,128))

# 
# img2 = cv2.rectangle(img, (130,180),(130+90,240), (0,0,0), -1) 
img2 = cv2.resize(crop_img,(96,96))
predictor = dlib.shape_predictor("shape_predictor_5_face_landmarks.dat")
tong = dlib.rectangle(0,0,128,128)
img_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

landmarks = predictor(img_gray,tong)


landmarks_list=[]
for i in range(0, landmarks.num_parts):
#     tong5 = str(landmarks.part(i).x), str(landmarks.part(i).y)
    landmarks_list.append((landmarks.part(i).x))
    landmarks_list.append((landmarks.part(i).y))
    
# img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
pixels = np.array(img2)
pixels = pixels.astype('float32')
pixels = pixels/255

img3 = pixels[None, :, :, :]
steering_angle = model.predict(img3)[0]
ttt = steering_angle*96
# for n in range(0, 5):
#     tttt = n*2
#     x = ttt[tttt]
#     y = ttt[tttt+1]
#     cv2.circle(img2, (x, y), 3, (255, 0, 0), -1)


x0 = (landmarks_list[0]+landmarks_list[2])/2
y0 = (landmarks_list[1]+landmarks_list[3])/2
x1 = (landmarks_list[4]+landmarks_list[6])/2
y1 = (landmarks_list[5]+landmarks_list[7])/2
x2 = float(landmarks_list[8])
y2 = float(landmarks_list[9])

cv2.circle(img2, (int(x0), int(y0)), 4, (255, 0, 0), -1)
cv2.circle(img2, (int(x1), int(y1)), 4, (255, 0, 0), -1)
cv2.circle(img2, (int(x2), int(y2)), 4, (255, 0, 0), -1)
image_widget.value = bgr8_to_jpeg(img2)

x0x1 = math.sqrt(math.pow(x0-x1, 2)+math.pow(y0-y1, 2))
x0x2 = math.sqrt(math.pow(x2-x0, 2)+math.pow(y2-y0, 2))
x1x2 = math.sqrt(math.pow(x2-x1, 2)+math.pow(y2-y1, 2))

# print()
# print(ttt)

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
teee = np.array([x0x1,x0x2,x1x2])
teee2 = np.array(ttt)
print(rmse(np.array(teee2), np.array(teee)))

3.3262484373034487


In [87]:
import math 

poly_cof = []
poly_cof.append([x0x1/128,x0x2/128,x1x2/128])

In [88]:
poly_cof

[[0.37743351028691136, 0.3114974151553067, 0.48937377773634083]]

In [84]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
teee = np.array([59,23,63])
teee2 = np.array([x0x1,x0x2,x1x2])
print(rmse(np.array(teee2), np.array(teee)))

In [85]:
teee,teee2

(array([59, 23, 63]), array([48.31148932, 39.87166914, 62.63984355]))

11.532955574646952
